<a href="https://colab.research.google.com/github/EsraCesur4/MusicEmotionRecognition/blob/main/Creating_Spotify_Datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Spotipy library

Spotipy is a lightweight Python library for the Spotify Web API. With Spotipy you get full access to all of the music data provided by the Spotify platform.
- GitHub: https://github.com/spotipy-dev/spotipy
- Documentation: https://spotipy.readthedocs.io/en/2.22.1/

Installing Spotipy:

In [ ]:
pip install spotipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 7.4 MB/s eta 0:00:00


## Drawing Song Information from Spotify Playlists

Here we can collect track features from Spotify playlists for different moods (e.g., angry, chill, happy, sad).

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy import util
import pandas as pd
import math
import pickle

features_for_mood = ['energy', 'liveness', 'tempo', 'speechiness',
                                     'acousticness', 'instrumentalness', 'danceability', 'duration_ms',
                                     'loudness', 'valence']

def get_track_features(track_ids, spotify):
    chunk_size = 50
    num_chunks = int(math.ceil(len(track_ids) / float(chunk_size)))
    features_add = []
    for i in range(num_chunks):
        chunk_track_ids = track_ids[i * chunk_size:min((i + 1) * chunk_size, len(track_ids))]
        chunk_features = spotify.audio_features(tracks=chunk_track_ids)
        features_add.extend(chunk_features)

    features_df = pd.DataFrame(features_add).drop(['id', 'analysis_url', 'key', 'mode', 'time_signature',
                                                   'track_href', 'type', 'uri'], axis=1)
    features_df = features_df[features_for_mood]
    return features_df

token = util.prompt_for_user_token(username='username', scope='user-library-read',
                                                   client_id='clientid',
                                                   client_secret='clientsecret',
                                                   redirect_uri='http://localhost:5000')
spotify = spotipy.Spotify(auth=token, requests_timeout=20)

playlists = {
  'angry' : ["https://open.spotify.com/playlist/37i9dQZF1EIgNZCaOGb0Mi?si=73bb7422787d46c1","https://open.spotify.com/playlist/37i9dQZF1EIcRK7JMCMZ3M?si=7498be7659024b5c"],
  'chill':    ["https://open.spotify.com/playlist/37i9dQZF1EVHGWrwldPRtj?si=50368e58043a4b7a","https://open.spotify.com/playlist/37i9dQZF1DX889U0CL85jj?si=aea9ecbcd9a7438b",],
  'happy':        ["https://open.spotify.com/playlist/7GhawGpb43Ctkq3PRP1fOL?si=0625e7c7914a4494","https://open.spotify.com/playlist/37i9dQZF1DXdPec7aLTmlC?si=714e88d0c5ec44ab",],
  'sad':  ["https://open.spotify.com/playlist/4yXfnhz0BReoVfwwYRtPBm?si=5f06cc8d9a884862","https://open.spotify.com/playlist/2Rvp6wYxx8Mznr35SsLLB8?si=37b65ab180304bec",]
}

tracks = pd.DataFrame()
moods = []

for mood, links in playlists.items():
    print (mood)
    for link in links:
        id = link[34:56]
        try:
            pl_tracks = spotify.playlist_tracks(id)['items']
            ids = [foo['track']['id'] for foo in pl_tracks]
        except:
            print (link)
            continue
        features = get_track_features(ids, spotify)
        features['id'] = ids
        features['mood'] = mood
        tracks = tracks.append(features)

tracks.to_csv('tracks.csv')